In [25]:
from bs4 import BeautifulSoup
import requests
import re
import requests
import pandas as pd

def search_company_page(company,market):
    #Given a company, find its webpage URL through the research system of Reuter

    #This URL return the search page for the given company
    search_url = f'https://www.reuters.com/finance/stocks/lookup?search={company}&searchType=any&sortBy=&dateRange=&comSortBy=marketcap'

    #Creating url request and bs4 object
    req=requests.get(search_url)
    
    soup = BeautifulSoup(req.content, "html.parser")

    #Searching for the table
    table_search = soup.findAll('tr')
    
    if market:
        #We explore the table
        for row in table_search:
            #If the stock market appears in the content of the row we select this row, and extract the url
            #located in "onclick", and stop the research
            if f"{market} Stock Exchange" in row.contents[-2]:
                company_page=row["onclick"].split("'")[1]
                break
        
        #we return the final url of the webpage of the company
    return re.sub(r'overview','financial-highlights',"https://www.reuters.com"+company_page)

def get_data(company,market):
    url=search_company_page(company,market)
    req=requests.get(url)
    soup = BeautifulSoup(req.content, "html.parser")

    ##Get stock price from company page
    price=float(soup.find("div",class_="sectionQuote nasdaqChange").
              findAll("span")[1].
              contents[0].
              strip()
             )

    ##Get %change from company page
    try:
        change = float(re.sub(r'^[(]|[%)]+$',"",
                          soup.
                          find("div",class_="sectionQuote priceChange").
                          findAll("span")[4].
                          contents[0].
                          strip()))/100
    except:
        change = re.sub(r'^[(]|[%)]+$',"",
                          soup.
                          find("div",class_="sectionQuote priceChange").
                          findAll("span")[4].
                          contents[0].
                          strip())

    try:
    ##Get Shares owned by institutionnal holders
        shared_owned, =(float(re.sub(r'^[(]|[%)]+$',"",elmt.parent.parent.find(class_="data").contents[0]))
                        for elmt in soup.findAll("strong") 
                        if "Shares Owned" in elmt.contents[0])
    except:
        shared_owned = "?"
    

    ##Get divident yield for company/industry/sector
    try:
        company_divident_yield, *rest = (float(elmt.parent.findAll(class_="data")[0].get_text())
                                   for elmt in soup.findAll("td") 
                                   if "Dividend Yield" in elmt.get_text())
    except:
        company_divident_yield = "?"
        
    try:    
        industry_divident_yield, *rest = (float(elmt.parent.findAll(class_="data")[1].get_text())
                                   for elmt in soup.findAll("td") 
                                   if "Dividend Yield" in elmt.get_text())
    except:
        industry_divident_yield = "?"

    try:    
        sector_divident_yield, *rest = (float(elmt.parent.findAll(class_="data")[2].get_text())
                                   for elmt in soup.findAll("td") 
                                   if "Dividend Yield" in elmt.get_text())
    except:
        sector_divident_yield = "?"
    
    try:
        sell_year_ending, *rest = (int(elmt.parent.findAll(class_="data")[0].get_text())
                                   for elmt in soup.findAll("td") 
                                   if "Quarter Ending" in elmt.get_text())
    except:
        sell_year_ending="?"

    return {"company":company,
            "price":price,
            "change":change,
            "shared_owned":shared_owned,
            "company_divident_yield":company_divident_yield,
            "industry_divident_yield":industry_divident_yield,
            "sell_year_ending":sell_year_ending}
        

In [26]:
reuter_L = []
company_list = ["Airbus","Danone","LVMH","total","edf","michelin","carrefour",'valeo','renault','peugeot']
market="Paris"
for company in company_list:
    reuter_L.append(get_data(company,market))

reuter_df = pd.DataFrame(reuter_L)


In [27]:
reuter_df

,change,company,company_divident_yield,industry_divident_yield,price,sell_year_ending,shared_owned
0,-0.0041,Airbus,1.45,1.34,102.08,5,43.53
1,-0.0056,Danone,2.49,2.35,65.27,5,50.60
2,-0.0431,LVMH,1.82,1.67,273.40,3,20.57
3,0.0159,total,5.42,4.43,55.70,3,40.05
4,-0.0013,edf,2.99,3.05,15.26,?,5.54
5,--,michelin,3.43,2.40,103.55,?,39.16
6,0.0098,carrefour,4.12,1.76,15.48,1,23.40
7,-0.025,valeo,1.99,1.53,31.64,1,54.67
8,-0.0041,renault,4.94,2.88,71.30,?,29.42
9,-0.0055,peugeot,2.44,2.88,21.53,1,36.40


NameError: name 'elmt' is not defined